__CORE REQUEST 2__

In [11]:
import pandas as pd
from datetime import datetime
import time
#import matplotlib.pyplot as plt
import json
import folium
from branca.colormap import linear
import branca.colormap as cm


In [13]:
geo_json_data = json.load(open('taxi_zones.json'))


In [14]:
# retieving data to be cleaned
raw_data  = pd.read_csv("yellow_tripdata_2018-01.csv")

# cleaning data:
# initial time equal final time
# distance <= 0
# total amount < 0 
# year not 2018
# month not January (change 1 with the appropiate month)
# trip duration > 200 minutes
# trip distance > 200 miles

raw_data['delta'] = (pd.to_datetime(raw_data['tpep_dropoff_datetime']) -  pd.to_datetime(raw_data['tpep_pickup_datetime'])).astype('timedelta64[m]')
data = raw_data.drop(raw_data[(raw_data.delta > 200) | (raw_data.trip_distance <= 0) | (raw_data.trip_distance > 200) | (raw_data.tpep_dropoff_datetime == raw_data.tpep_pickup_datetime) | (raw_data.total_amount < 0) | (pd.DatetimeIndex(raw_data['tpep_pickup_datetime']).month != 1) | (pd.DatetimeIndex(raw_data['tpep_pickup_datetime']).year != 2018)].index) 

# free memory deleting the raw_data
del raw_data

data = data.loc[:, ['PULocationID', 'DOLocationID']]

data['PULocationID_2'] = data['PULocationID']
data_pu = data.set_index('PULocationID')['PULocationID_2']

data['DOLocationID_2'] = data['DOLocationID']
data_do = data.set_index('DOLocationID')['DOLocationID_2']
del data

In [15]:
for i in range(2,7):
        # retieving data to be cleaned
        raw_data  = pd.read_csv("yellow_tripdata_2018-0"+str(i)+".csv")

        # cleaning data
        raw_data['delta'] = (pd.to_datetime(raw_data['tpep_dropoff_datetime']) -  pd.to_datetime(raw_data['tpep_pickup_datetime'])).astype('timedelta64[m]')
        data_i = raw_data.drop(raw_data[(raw_data.delta > 200) | (raw_data.trip_distance <= 0) | (raw_data.trip_distance > 200) | (raw_data.tpep_dropoff_datetime == raw_data.tpep_pickup_datetime) | (raw_data.total_amount < 0) | (pd.DatetimeIndex(raw_data['tpep_pickup_datetime']).month != i) | (pd.DatetimeIndex(raw_data['tpep_pickup_datetime']).year != 2018)].index) 

        # free memory deleting the raw_data
        del raw_data
        
        
        data_i = data_i.loc[:, ['PULocationID', 'DOLocationID']]
        #data = data.append(data_i)
        
        data_i['PULocationID_2'] = data_i['PULocationID']
        data_pu = data_pu.append(data_i.set_index('PULocationID')['PULocationID_2'])

        data_i['DOLocationID_2'] = data_i['DOLocationID']
        data_do = data_do.append(data_i.set_index('DOLocationID')['DOLocationID_2'])
        del data_i
        
pu = pd.DataFrame(data_pu.value_counts())
pu_dict = pu.set_index(pu.index)['PULocationID_2']
do = pd.DataFrame(data_do.value_counts())
do_dict = do.set_index(do.index)['DOLocationID_2']

del data_pu
del data_do

In [26]:
linear_pu = cm.LinearColormap(
    ['green', 'yellow', 'red'],
    vmin=pu.tail(1).PULocationID_2.item(), vmax=pu.head(1).PULocationID_2.item(),
    caption= 'Taxi pickup zone scale (number of rides)'
)



def daje_pu(feature):
    try:
        return pu_dict[feature['properties']['LocationID']]
    except:
        return 176
    
m = folium.Map([40.7, -74], tiles='cartodbpositron', zoom_start=10.5)

folium.GeoJson(
    geo_json_data,
    style_function=lambda feature: {
        'fillColor': linear_pu(daje_pu(feature)),
        'color': 'black',
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }
).add_to(m)
linear_pu.add_to(m)

m.save('pickup.html')
m.render

<bound method Map.render of <folium.folium.Map object at 0x000002457ADD2278>>

In [25]:
linear_do = cm.LinearColormap(
    ['green', 'yellow', 'red'],
    vmin=do.tail(1).DOLocationID_2.item(), vmax=do.head(1).DOLocationID_2.item(),
    caption= 'Taxi dropoff zone scale (number of rides)'
)

def daje_do(feature):
    try:
        return do_dict[feature['properties']['LocationID']]
    except:
        return 2
    
n = folium.Map([40.7, -74], tiles='cartodbpositron', zoom_start=10.5)

folium.GeoJson(
    geo_json_data,
    style_function=lambda feature: {
        'fillColor': linear_do(daje_do(feature)),
        'color': 'black',
        'dashArray': '5, 5',
        'fillOpacity': 0.9
    }
).add_to(n)
linear_do.add_to(n)


n.save('dropoff.html')
n.render

<bound method Map.render of <folium.folium.Map object at 0x000002457ADD2240>>